In [1]:
import sys
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras import layers
from numpy import array
from pickle import dump
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ReduceLROnPlateau


Using TensorFlow backend.


In [2]:
# import tensorflow as tf
# config = tf.ConfigProto(allow_soft_placement=True)
# config.gpu_options.allow_growth=True

In [3]:
# проверяем, какая видюха, результаты в черном окошке с юпитером
## Creates a graph.
# a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
# b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
# c = tf.matmul(a, b)
# # Creates a session with log_device_placement set to True.
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# # Runs the op.
# print(sess.run(c))

In [2]:
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r',encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [3]:
in_filename = 'gk4z_df_10-20_TXT.txt'
data = load_doc(in_filename)

In [4]:
# tokenizer = Tokenizer()
tokenizer = load(open('tokenizer_gen_7_TOKENS10-20PADDING.pkl',  'rb'))

In [5]:

corpus = data.split("\n")    
# tokenizer.fit_on_texts(corpus)
vocab_size = len(tokenizer.word_index) + 1

In [6]:
print('vocab_size: ', vocab_size)
print('number of lines: ', len(corpus))
print('all the words: ', len(data.split()))

vocab_size:  19794
number of lines:  7206
all the words:  101689


In [7]:
input_sequences = []
for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range (1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)


In [8]:
max_sequence_len = max([len(x) for x in input_sequences])
max_sequence_len

19

In [9]:
input_sequences = array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [10]:
input_sequences.shape

(94483, 19)

In [11]:
input_sequences

array([[    0,     0,     0, ...,     0,  1494,  7043],
       [    0,     0,     0, ...,  1494,  7043,  7044],
       [    0,     0,     0, ...,  7043,  7044,    47],
       ...,
       [    0,     0,     0, ...,  1287,   719,    34],
       [    0,     0,     0, ...,   719,    34,     1],
       [    0,     0,     0, ...,    34,     1, 19793]])

In [12]:

X,y = input_sequences[:,:-1],input_sequences[:,-1]


In [13]:
y = np_utils.to_categorical(y, num_classes=vocab_size)

In [14]:
y.shape

(94483, 19794)

In [15]:
seq_length = X.shape[1]
seq_length

18

In [16]:
model = load_model('model__gen_7_TOKENS10-20PADDING.h5')
model.load_weights('weights-TOK10-20PAD_gen7_improv-38-2.8597.hdf5')


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


The learned embedding needs to know the size of the vocabulary and the length of input sequences as previously discussed. It also has a parameter to specify how many dimensions will be used to represent each word. That is, the size of the embedding vector space.

Common values are 50, 100, and 300. We will use 50 here, but consider testing smaller or larger values.

We will use a two LSTM hidden layers with 100 memory cells each. More memory cells and a deeper network may achieve better results.

A dense fully connected layer with 100 neurons connects to the LSTM hidden layers to interpret the features extracted from the sequence. The output layer predicts the next word as a single vector the size of the vocabulary with a probability for each word in the vocabulary. A softmax activation function is used to ensure the outputs have the characteristics of normalized probabilities.

In [17]:
# model = Sequential()
# model.add(layers.Embedding(vocab_size, 300, input_length=seq_length))
# model.add(LSTM(300, return_sequences=True))
# model.add(Dropout(0.1))
# model.add(LSTM(300, return_sequences=True))
# model.add(Dropout(0.1))
# model.add(LSTM(300))
# model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 18, 300)           5938200   
_________________________________________________________________
lstm_1 (LSTM)                (None, 18, 300)           721200    
_________________________________________________________________
dropout_1 (Dropout)          (None, 18, 300)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 18, 300)           721200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 18, 300)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dense_1 (Dense)              (None, 19794)             5957994   
Total para

In [18]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [19]:
filepath="weights-TOK10-20PAD_gen7_improv-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [20]:

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1,  min_lr=0.000001, verbose=1,cooldown=0)
callbacks_list = [checkpoint, reduce_lr]

In [21]:
model.fit(X, y, epochs=100, callbacks=callbacks_list)

Epoch 1/100
94483/94483 [==============================] - 369s 4ms/step - loss: 3.1258

Epoch 00001: loss improved from inf to 3.12577, saving model to weights-TOK10-20PAD_gen7_improv-01-3.1258.hdf5
Epoch 2/100
94483/94483 [==============================] - 363s 4ms/step - loss: 3.1697

Epoch 00002: loss did not improve from 3.12577

Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 3/100
94483/94483 [==============================] - 363s 4ms/step - loss: 3.0333

Epoch 00003: loss improved from 3.12577 to 3.03334, saving model to weights-TOK10-20PAD_gen7_improv-03-3.0333.hdf5
Epoch 4/100
94483/94483 [==============================] - 362s 4ms/step - loss: 2.9764

Epoch 00004: loss improved from 3.03334 to 2.97644, saving model to weights-TOK10-20PAD_gen7_improv-04-2.9764.hdf5
Epoch 5/100
94483/94483 [==============================] - 364s 4ms/step - loss: 2.9565

Epoch 00005: loss improved from 2.97644 to 2.95652, saving model to weights-TOK10-20P


Epoch 00044: loss did not improve from 2.84325
Epoch 45/100
94483/94483 [==============================] - 363s 4ms/step - loss: 2.8447

Epoch 00045: loss did not improve from 2.84325
Epoch 46/100
94483/94483 [==============================] - 362s 4ms/step - loss: 2.8444

Epoch 00046: loss did not improve from 2.84325
Epoch 47/100
94483/94483 [==============================] - 362s 4ms/step - loss: 2.8449

Epoch 00047: loss did not improve from 2.84325
Epoch 48/100
94483/94483 [==============================] - 363s 4ms/step - loss: 2.8444

Epoch 00048: loss did not improve from 2.84325
Epoch 49/100
94483/94483 [==============================] - 361s 4ms/step - loss: 2.8456

Epoch 00049: loss did not improve from 2.84325
Epoch 50/100
94483/94483 [==============================] - 363s 4ms/step - loss: 2.8434

Epoch 00050: loss did not improve from 2.84325
Epoch 51/100
94483/94483 [==============================] - 362s 4ms/step - loss: 2.8428

Epoch 00051: loss improved from 2.84325 

In [24]:
# save the model to file
model.save('model__gen_7_TOKENS10-20PADDING.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer_gen_7_TOKENS10-20PADDING.pkl', 'wb'))

In [22]:
def generate_text(seed_text, n_words, max_sequence_len, model):
#     print(seed_text)
    result = ''
    for j in range(n_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
#         print('token_list ', token_list)
        token_list = pad_sequences([token_list], maxlen= 
                             max_sequence_len-1, padding='pre')
#         print('pad_token_list ', token_list)
        predicted = model.predict_classes(token_list, verbose=0)
#         print('predicted ', predicted)
  
        output_word = ""
        
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                result += " " + output_word
#                 print('output_word ', output_word)
                break
        seed_text += " " + output_word
        
#         print('seed_text ', seed_text)
    return result

In [23]:
for i in range (10):
  seed_text = corpus[randint(0,len(corpus))]
  print(seed_text)
  generated = generate_text(seed_text, 10,  max_sequence_len, model)
  print(generated+ '\n')

у второй половинки чгкз находится на первом месте в списке пабликов считается ли это за идеальный выбор человека
 ночь бабла власти ебанутые эффектами умеют выход концертов президент приехал

спасибо что без мьюз и аймэикс не гоню на альбомы просто достали в каждый топ их включать
 все плейлиста класс земфира стоило приделывать будет сообщений россию ночь

заебись когда показываешь басисту что хочешь пить и он даёт тебе водички
 музыкант не хватает а друг тут квартетов лол лол мои

чем больше концертов посещаю тем больше песен не могу слушать потому что воспоминания нормально ли это
 за сына встречаться нужно добавил библии так купить спокойно президент

5 сентября пойти на аэросмит на лубянке или на макса коржа в лужниках
 клуб только пытаться делаю в инстаграм чтобы обезопасит купить обезопасит

в этот день 5 лет назад весь мир узнал как сильно можно любить группу
 чтобы запись присылают не надоела разножанровые понимаю ждём концерт концерт

кто выпил бухло спрятанное около арены вып

In [ ]:
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)